RUNALL

In [10]:
import nest_asyncio
nest_asyncio.apply()

import os, shutil
from googletrans import Translator
from bs4 import BeautifulSoup
import requests
from gtts import gTTS
from pathlib import Path
import tkinter as tk
from tkinter import *
from tkinter import ttk
import asyncio
import threading

deck_name = "Testing Deck"
global standard_deck_name, known_lang, target_lang, front_format, back_format
standard_deck_name = deck_name.lower().replace(" ","_")
known_lang = "es"
target_lang = "fr"
front_format = "[AUDIO_TARGET_LANG]<br>[IMAGE]"
back_format = "[TEXT_TARGET_LANG]<br>([TEXT_KNOWN_LANG])"

global card_list
card_list = []

In [11]:

async def translate_text(text):
     async with Translator() as translator:
        textdict = {}
        language = await translator.detect(text)
        if language.confidence < 0.1 or language.lang not in [known_lang, target_lang]:
                print(f"WARNING: Couldn't detect language at {text}! Assuming text is in {known_lang}.")
                translated = await translator.translate(text, src=known_lang, dest=target_lang)
                textdict[known_lang] = text
                textdict[target_lang] = translated.text
                return textdict
        elif language.lang == known_lang:
                translated = await translator.translate(text, src=known_lang, dest=target_lang)
                textdict[known_lang] = text
                textdict[target_lang] = translated.text
                return textdict
        elif language.lang == target_lang:
                translated = await translator.translate(text, src=target_lang, dest=known_lang)
                textdict[known_lang] = translated.text
                textdict[target_lang] = text
                return textdict
        else:
                print(f"Error when translating: {text}")
                return -1
        
def download_image(word):
        url = f'https://www.google.com/search?q={word}&source=lnms&tbm=isch&tbs=isz:m'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        imgs = soup.find_all("img")
        img_urls = [img["src"] for img in imgs if "src" in img.attrs]

        for img_url in img_urls[1:4]:
                try:
                        img_response = requests.get(img_url)
                        media_path = os.path.join(os.getenv("APPDATA"),"Anki2", "User 1", "collection.media")
                        img_path = os.path.join(media_path, f"{standard_deck_name}-{word}.jpg")
                        with open(img_path, "wb") as f:
                                f.write(img_response.content)
                        return f'<img src="{standard_deck_name}-{word}.jpg">'
                except Exception as e:
                        print(f"Error downloading image: {e}")
        return -1

def download_audio(word, lang):
        speech = gTTS(word, lang=lang)
        speech.save(f"{standard_deck_name}-{word}-{lang}.mp3")
        source_path = os.path.join(os.getcwd(), f"{standard_deck_name}-{word}-{lang}.mp3")
        media_path = os.path.join(os.getenv("APPDATA"),"Anki2", "User 1", "collection.media")
        destination_path = os.path.join(media_path, f"{standard_deck_name}-{word}-{lang}.mp3")
        
        shutil.move(source_path, destination_path)
        return f'[sound:{standard_deck_name}-{word}-{lang}.mp3]'

def fill_details(text, front_format, back_format):
        full_query = front_format + ";" + back_format
        card_details = {}
        
        loop = asyncio.get_event_loop()
        textdict = loop.run_until_complete(translate_text(text))
        #loop = asyncio.get_event_loop()
        #task = asyncio.ensure_future(translate_text(text))
        #textdict = loop.run_until_complete(task)
        
        card_details["text_known_lang"] = textdict[known_lang]
        card_details["text_target_lang"] = textdict[target_lang]
        if "[AUDIO_KNOWN_LANG]" in full_query:
                card_details["audio_known_lang"] = download_audio(textdict[known_lang], known_lang)
        else:
                card_details["audio_known_lang"] = ""
        if "[AUDIO_TARGET_LANG]" in full_query:
                card_details["audio_target_lang"] = download_audio(textdict[target_lang], target_lang)
        else:
                card_details["audio_target_lang"] = ""
        if "[IMAGE]" in full_query:
                card_details["image"] = download_image(text)
        else:
                card_details["image"] = ""
        card_list.append(card_details)
        return card_details

def add_line(card_details, front_format, back_format):
        map = {
                "[TEXT_KNOWN_LANG]":card_details["text_known_lang"],
                "[TEXT_TARGET_LANG]":card_details["text_target_lang"],
                "[AUDIO_KNOWN_LANG]":card_details["audio_known_lang"],
                "[AUDIO_TARGET_LANG]":card_details["audio_target_lang"],
                "[IMAGE]":card_details["image"],
                "\n":"<br>"
        }
        for decorator in map:
                front_format = front_format.replace(decorator, map[decorator])
                back_format = back_format.replace(decorator, map[decorator])
        return f'{standard_deck_name}	{front_format}	{back_format}\n'

In [14]:
loop = asyncio.get_event_loop()
textdict = loop.run_until_complete(translate_text("el gato"))

AsyncLibraryNotFoundError: unknown async library, or not in async context

In [ ]:
class App:
    def __init__(self, root):
        self.root = root
        self.root.title("FastFlashcards v0.1")
        self.root.geometry("500x600")
        
        self.create_menu()
        
    
    
    def create_menu(self):
        #Adds menu bar on top of app
        menu = Menu(self.root)
        options_tab = Menu(menu)
        options_tab.add_command(label="Option 1")
        options_tab.add_command(label="Option 2")
        options_tab.add_command(label="exit", command=self.root.destroy)
        menu.add_cascade(label="Options", menu=options_tab)
        self.root.config(menu=menu)
    
    def main_widgets(self):
        #Adds widgets
        self.title = tk.Label(self.root, text="FastFlashcards", font=("arial",30)).pack(pady=25)
        
        self.kl_frame = tk.Frame(self.root)
        self.kl = tk.StringVar()
        tk.Label(self.kl_frame, text="Known Language: ").pack(side=tk.LEFT)
        tk.Entry(self.kl_frame, textvariable=self.kl).pack(side=tk.LEFT)
        self.kl_frame.pack()
        self.tl_frame = tk.Frame(self.root)
        self.tl = StringVar()
        tk.Label(self.tl_frame, text="Target Language: ").pack(side=tk.LEFT)
        tk.Entry(self.tl_frame, textvariable=self.tl).pack(side=tk.LEFT)
        self.tl_frame.pack()
        
        tk.Label(self.root, text="Enter text to make card!").pack(pady=5)
        self.entry_frame = tk.Frame(self.root)
        self.card_text = tk.StringVar()
        ttk.Entry(self.entry_frame, textvariable=self.card_text).pack(side=tk.LEFT, padx=5)
        self.cards_added = 0
        self.added_cards_strvar = tk.StringVar()
        self.added_cards_strvar.set("")
        tk.Button(self.entry_frame, text="Create Card", command=self.upload_text).pack()
        self.entry_frame.pack()
        tk.Label(self.root, textvariable=self.added_cards_strvar).pack()
        tk.Button(self.root, text="Export TXT File", command=self.make_txt).pack(pady=15)
        
    def upload_text(self):
        #Makes card?
        text = self.card_text.get()
        if text == "":
            return
        card_details = fill_details(text, front_format, back_format)
        
        self.card_text.set("")
        self.cards_added += 1
        self.added_cards_strvar.set(f"Added {self.cards_added} cards to deck.")
        return card_details
    
    def make_txt(self):
        initial_dir = os.getcwd()
        os.chdir(os.path.join(initial_dir, "decks"))
        txt_path = os.path.join(os.getcwd(), f"{standard_deck_name}.txt")
        
        if not os.path.isfile(txt_path):
            with open(f"{standard_deck_name}.txt", "w") as txt:
                txt.write("#separator:tab\n")
                txt.write("#html:true\n")
                txt.write("#deck column: 1\n")
                txt.write("#tags column: 4\n")
                txt.close()
        
        txt = open(f"{standard_deck_name}.txt", "a")
        print(".txt file opened")
        for card in card_list:
            txt.write(line(card, front_format, back_format))
        txt.close()
        os.chdir("..")
        

In [13]:
root = tk.Tk()
app = App(root)
app.main_widgets()
app.root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Gamer\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\httpcore\_async\connection_pool.py", line 228, in handle_async_request
    closing = self._assign_requests_to_connections()
  File "c:\Users\Gamer\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\httpcore\_async\connection_pool.py", line 328, in _assign_requests_to_connections
    pool_request.assign_to_connection(connection)
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
  File "c:\Users\Gamer\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\httpcore\_async\connection_pool.py", line 25, in assign_to_connection
    self._connection_acquired.set()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "c:\Users\Gamer\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\httpcore\_synchronization.py", line 130, in set
    self.setup()
    ~~~~~~~~~~^^
  File "c:\Users\Gamer\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\

KeyboardInterrupt: 